In [1]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import mysql.connector

# Step 1: Connect without selecting a database
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=""
)

cursor = mydb.cursor()

# Step 2: Create database if not exists
cursor.execute("CREATE DATABASE IF NOT EXISTS tennis_analytics;")

# Step 3: Now connect to the new database
mydb.database = 'tennis_analytics'

In [9]:
# 1.Categories Table and Competitions Table
import requests
import mysql.connector
import json

# === CONFIGURATION ===
API_KEY = "rltR6inycjP6oVTtY8lPaCd10gyZDchOu51tA3lo"  # Replace with your actual API key
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',       # Replace with your DB username
    'password': '',   # Replace with your DB password
    'database': 'tennis_analytics'      # Replace with your DB name
}

# === 1. FETCH DATA FROM API ===
url = f"https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key={API_KEY}"
response = requests.get(url)

if response.status_code != 200:
    print("Error fetching data:", response.text)
    exit()

data = response.json()

# Optional: Uncomment to inspect the full structure
# print(json.dumps(data, indent=2))

# === 2. EXTRACT CATEGORIES AND COMPETITIONS ===
categories = {}
competitions = []

for comp in data.get('competitions', []):
    cat = comp.get('category', {})
    category_id = cat.get('id')
    category_name = cat.get('name')

    if category_id and category_name:
        categories[category_id] = category_name

    competitions.append({
        'competition_id': comp['id'],
        'competition_name': comp['name'],
        'parent_id': comp.get('parent_id'),
        'type': comp['type'],
        'gender': comp['gender'],
        'category_id': category_id
    })

# === 3. CONNECT TO MYSQL ===
conn = mysql.connector.connect(**DB_CONFIG)
cursor = conn.cursor()

# === 4. CREATE TABLES ===
cursor.execute("""
CREATE TABLE IF NOT EXISTS Categories (
    category_id VARCHAR(50) PRIMARY KEY,
    category_name VARCHAR(100) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Competitions (
    competition_id VARCHAR(50) PRIMARY KEY,
    competition_name VARCHAR(100) NOT NULL,
    parent_id VARCHAR(50),
    type VARCHAR(20) NOT NULL,
    gender VARCHAR(10) NOT NULL,
    category_id VARCHAR(50),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);
""")

# === 5. INSERT DATA ===
for cid, cname in categories.items():
    cursor.execute("""
        INSERT IGNORE INTO Categories (category_id, category_name)
        VALUES (%s, %s)
    """, (cid, cname))

for comp in competitions:
    cursor.execute("""
        INSERT IGNORE INTO Competitions (
            competition_id, competition_name, parent_id, type, gender, category_id
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        comp['competition_id'],
        comp['competition_name'],
        comp['parent_id'],
        comp['type'],
        comp['gender'],
        comp['category_id']
    ))

conn.commit()

# === 6. RUN ANALYSIS QUERIES ===
queries = {
    "All competitions with category name": """
        SELECT c.competition_name, cat.category_name
        FROM Competitions c
        JOIN Categories cat ON c.category_id = cat.category_id;
    """,

    "Count of competitions per category": """
        SELECT cat.category_name, COUNT(*) AS competition_count
        FROM Competitions c
        JOIN Categories cat ON c.category_id = cat.category_id
        GROUP BY cat.category_name;
    """,

    "Competitions of type 'doubles'": """
        SELECT competition_name
        FROM Competitions
        WHERE type = 'doubles';
    """,

    "Competitions in 'ITF Men' category": """
        SELECT c.competition_name
        FROM Competitions c
        JOIN Categories cat ON c.category_id = cat.category_id
        WHERE cat.category_name = 'ITF Men';
    """,

    "Parent and sub-competitions": """
        SELECT parent.competition_name AS parent_name, child.competition_name AS sub_competition
        FROM Competitions child
        JOIN Competitions parent ON child.parent_id = parent.competition_id;
    """,

    "Competition type distribution by category": """
        SELECT cat.category_name, c.type, COUNT(*) AS type_count
        FROM Competitions c
        JOIN Categories cat ON c.category_id = cat.category_id
        GROUP BY cat.category_name, c.type;
    """,

    "Top-level competitions (no parent)": """
        SELECT competition_name
        FROM Competitions
        WHERE parent_id IS NULL;
    """
}
print("\n=== SQL Query Results ===\n")
for title, query in queries.items():
    print(f"\n-- {title} --")
    cursor.execute(query)
    rows = cursor.fetchall()
    for row in rows:
        print(row)
# === CLEANUP ===
cursor.close()
conn.close()


=== SQL Query Results ===


-- All competitions with category name --
('IPTL', 'IPTL')
('Juniors AO, Melbourne, Australia Men Singles', 'Juniors')
('Juniors AO, Melbourne, Australia Men Doubles', 'Juniors')
('Juniors AO, Melbourne, Australia Women Singles', 'Juniors')
('Juniors AO, Melbourne, Australia Women Doubles', 'Juniors')
('Juniors French Open, Paris, France Men Singles', 'Juniors')
('Juniors French Open, Paris, France Men Doubles', 'Juniors')
('Juniors French Open, Paris, France Women Singles', 'Juniors')
('Juniors French Open, Paris, France Women Doubles', 'Juniors')
('Juniors Wimbledon, London, GB Women Singles', 'Juniors')
('Juniors Wimbledon, London, GB Men Singles', 'Juniors')
('Juniors Wimbledon, London, GB Men Doubles', 'Juniors')
('Juniors Wimbledon, London, GB Women Doubles', 'Juniors')
('Juniors US Open, New York, USA Men Singles', 'Juniors')
('Juniors US Open, New York, USA Men Doubles', 'Juniors')
('Juniors US Open, New York, USA Women Singles', 'Juniors')
('Junior

In [8]:
# competition_queries.py

import mysql.connector
import pandas as pd

# === UPDATE THIS WITH YOUR DB CREDENTIALS ===
DB_CONFIG = {
    'host':     'localhost',
    'user':     'root',           # your DB username
    'password': '',               # your DB password
    'database': 'tennis_analytics'
}

def _run_query(sql: str) -> pd.DataFrame:
    """Helper: connect, run SQL, return DataFrame."""
    conn   = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute(sql)
    rows    = cursor.fetchall()
    cols    = [desc[0] for desc in cursor.description]
    df      = pd.DataFrame(rows, columns=cols)
    cursor.close()
    conn.close()
    return df


def list_competitions_with_category() -> pd.DataFrame:
    """1. List all competitions along with their category name."""
    sql = """
        SELECT c.competition_name,
               cat.category_name
        FROM Competitions AS c
        JOIN Categories    AS cat
          ON c.category_id = cat.category_id;
    """
    return _run_query(sql)


def count_competitions_per_category() -> pd.DataFrame:
    """2. Count the number of competitions in each category."""
    sql = """
        SELECT cat.category_name,
               COUNT(*) AS competition_count
        FROM Competitions AS c
        JOIN Categories    AS cat
          ON c.category_id = cat.category_id
        GROUP BY cat.category_name;
    """
    return _run_query(sql)


def find_doubles_competitions() -> pd.DataFrame:
    """3. Find all competitions of type 'doubles'."""
    sql = """
        SELECT competition_name
        FROM Competitions
        WHERE type = 'doubles';
    """
    return _run_query(sql)


def get_competitions_in_itf_men() -> pd.DataFrame:
    """4. Get competitions that belong to the 'ITF Men' category."""
    sql = """
        SELECT c.competition_name
        FROM Competitions AS c
        JOIN Categories    AS cat
          ON c.category_id = cat.category_id
        WHERE cat.category_name = 'ITF Men';
    """
    return _run_query(sql)


def parent_and_sub_competitions() -> pd.DataFrame:
    """5. Identify parent competitions and their sub‑competitions."""
    sql = """
        SELECT parent.competition_name AS parent_name,
               child.competition_name  AS sub_competition_name
        FROM Competitions AS child
        JOIN Competitions AS parent
          ON child.parent_id = parent.competition_id;
    """
    return _run_query(sql)


def competition_type_distribution() -> pd.DataFrame:
    """6. Analyze the distribution of competition types by category."""
    sql = """
        SELECT cat.category_name,
               c.type,
               COUNT(*) AS type_count
        FROM Competitions AS c
        JOIN Categories    AS cat
          ON c.category_id = cat.category_id
        GROUP BY cat.category_name, c.type;
    """
    return _run_query(sql)


def list_top_level_competitions() -> pd.DataFrame:
    """7. List all competitions with no parent (top‑level)."""
    sql = """
        SELECT competition_name
        FROM Competitions
        WHERE parent_id IS NULL;
    """
    return _run_query(sql)


if __name__ == "__main__":
    # Example usage: print each result to console
    print("\n1. Competitions with Category:\n", list_competitions_with_category())
    print("\n2. Count per Category:\n",    count_competitions_per_category())
    print("\n3. Doubles Competitions:\n",    find_doubles_competitions())
    print("\n4. ITF Men Competitions:\n",     get_competitions_in_itf_men())
    print("\n5. Parent ↔ Sub Competitions:\n", parent_and_sub_competitions())
    print("\n6. Type Distribution:\n",        competition_type_distribution())
    print("\n7. Top‑Level Competitions:\n",    list_top_level_competitions())



1. Competitions with Category:
                                     competition_name category_name
0                                               IPTL          IPTL
1       Juniors AO, Melbourne, Australia Men Singles       Juniors
2       Juniors AO, Melbourne, Australia Men Doubles       Juniors
3     Juniors AO, Melbourne, Australia Women Singles       Juniors
4     Juniors AO, Melbourne, Australia Women Doubles       Juniors
...                                              ...           ...
6017             WTA 125K Newport, USA Women Singles      WTA 125K
6018   WTA 125K Taipei, Chinese Taipei Women Singles      WTA 125K
6019   WTA 125K Taipei, Chinese Taipei Women Doubles      WTA 125K
6020           WTA 125K Cali, Colombia Women Singles      WTA 125K
6021           WTA 125K Cali, Colombia Women Doubles      WTA 125K

[6022 rows x 2 columns]

2. Count per Category:
            category_name  competition_count
0                    ATP                223
1   Billie Jean King Cup 

In [11]:
# 2.Complexes Table and Venues table :
import requests
import mysql.connector

# === 1. API Configuration ===
API_KEY = 'rltR6inycjP6oVTtY8lPaCd10gyZDchOu51tA3lo'
API_URL = f"https://api.sportradar.com/tennis/trial/v3/en/complexes.json?api_key={API_KEY}"

# === 2. MySQL Database Connection ===
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tennis_analytics'
)
cursor = conn.cursor()

# === 3. Create Tables ===
cursor.execute("""
CREATE TABLE IF NOT EXISTS Complexes (
    complex_id VARCHAR(50) PRIMARY KEY,
    complex_name VARCHAR(100) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Venues (
    venue_id VARCHAR(50) PRIMARY KEY,
    venue_name VARCHAR(100) NOT NULL,
    city_name VARCHAR(100) NOT NULL,
    country_name VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    timezone VARCHAR(100) NOT NULL,
    complex_id VARCHAR(50),
    FOREIGN KEY (complex_id) REFERENCES Complexes(complex_id)
);
""")

conn.commit()

# === 4. Fetch Data from API ===
response = requests.get(API_URL)
data = response.json()

# === 5. Insert Data into Tables ===
for complex_ in data.get('complexes', []):
    complex_id = complex_['id']
    complex_name = complex_['name']

    cursor.execute("""
        INSERT IGNORE INTO Complexes (complex_id, complex_name)
        VALUES (%s, %s)
    """, (complex_id, complex_name))

    for venue in complex_.get('venues', []):
        venue_id = venue['id']
        venue_name = venue['name']
        city_name = venue['city_name']
        country_name = venue['country_name']
        country_code = venue['country_code']
        timezone = venue['timezone']

        cursor.execute("""
            INSERT IGNORE INTO Venues (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (venue_id, venue_name, city_name, country_name, country_code, timezone, complex_id))

conn.commit()

# === 6. Run SQL Queries ===

def run_query(title, query):
    print(f"\n--- {title} ---")
    cursor.execute(query)
    for row in cursor.fetchall():
        print(row)

# a. List all venues along with their associated complex name
run_query("All Venues with Complex Name", """
    SELECT v.venue_name, c.complex_name
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
""")

# b. Count the number of venues in each complex
run_query("Venue Count per Complex", """
    SELECT c.complex_name, COUNT(v.venue_id) AS venue_count
    FROM Complexes c
    JOIN Venues v ON c.complex_id = v.complex_id
    GROUP BY c.complex_name
""")

# c. Get details of venues in a specific country (Chile)
run_query("Venues in Chile", """
    SELECT venue_name, city_name, country_name
    FROM Venues
    WHERE country_name = 'Chile'
""")

# d. Identify all venues and their timezones
run_query("Venues and Timezones", """
    SELECT venue_name, timezone
    FROM Venues
""")

# e. Complexes with more than one venue
run_query("Complexes with > 1 Venue", """
    SELECT c.complex_name, COUNT(v.venue_id) AS venue_count
    FROM Complexes c
    JOIN Venues v ON c.complex_id = v.complex_id
    GROUP BY c.complex_name
    HAVING COUNT(v.venue_id) > 1
""")

# f. List venues grouped by country
run_query("Venues Grouped by Country", """
    SELECT country_name, COUNT(venue_id) AS venue_count
    FROM Venues
    GROUP BY country_name
""")

# g. Find all venues for a specific complex (e.g., Nacional)
run_query("Venues for Complex 'Nacional'", """
    SELECT v.venue_name
    FROM Venues v
    JOIN Complexes c ON v.complex_id = c.complex_id
    WHERE c.complex_name = 'Nacional'
""")

# === 7. Close Connection ===
cursor.close()
conn.close()



--- All Venues with Complex Name ---
('Cancha Central', 'Nacional')
('Cancha Central', 'Nacional')
('Centre Court', 'Estadio de la Cartuja')
('Centre Court', 'Estadio de la Cartuja')
('Court One', 'Estadio de la Cartuja')
('Court One', 'Estadio de la Cartuja')
('COURT 1', 'Sibur Arena')
('COURT 1', 'Sibur Arena')
('CENTER COURT', 'Sibur Arena')
('CENTER COURT', 'Sibur Arena')
('Sibur Arena', 'Sibur Arena')
('Sibur Arena', 'Sibur Arena')
('TC Dynamo', 'Sibur Arena')
('TC Dynamo', 'Sibur Arena')
('Central', 'Complexo de Tenis do Jamor')
('Central', 'Complexo de Tenis do Jamor')
('Campo 3', 'Complexo de Tenis do Jamor')
('Campo 3', 'Complexo de Tenis do Jamor')
('Campo 4', 'Complexo de Tenis do Jamor')
('Campo 4', 'Complexo de Tenis do Jamor')
('Campo 1', 'Complexo de Tenis do Jamor')
('Campo 1', 'Complexo de Tenis do Jamor')
('Campo 8', 'Complexo de Tenis do Jamor')
('Campo 8', 'Complexo de Tenis do Jamor')
('Centre Court', 'Ostravar Arena')
('Centre Court', 'Ostravar Arena')
('Court 3'

In [13]:
# venue_queries.py

import mysql.connector
import pandas as pd

# === 0. UPDATE WITH YOUR DATABASE CREDENTIALS ===
DB_CONFIG = {
    'host':     'localhost',
    'user':     'root',           # your DB username
    'password': '',               # your DB password
    'database': 'tennis_analytics'
}

def _run_query(sql: str, params: tuple = ()) -> pd.DataFrame:
    """Helper: connect, run the given SQL (with optional params), return a DataFrame."""
    conn   = mysql.connector.connect(**DB_CONFIG)
    cursor = conn.cursor()
    cursor.execute(sql, params)
    rows   = cursor.fetchall()
    cols   = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return pd.DataFrame(rows, columns=cols)


def list_venues_with_complex_name() -> pd.DataFrame:
    """
    1. List all venues along with their associated complex name.
    """
    sql = """
        SELECT v.venue_name,
               c.complex_name
        FROM Venues    AS v
        JOIN Complexes AS c
          ON v.complex_id = c.complex_id;
    """
    return _run_query(sql)


def count_venues_per_complex() -> pd.DataFrame:
    """
    2. Count the number of venues in each complex.
    """
    sql = """
        SELECT c.complex_name,
               COUNT(v.venue_id) AS venue_count
        FROM Complexes AS c
        JOIN Venues    AS v
          ON c.complex_id = v.complex_id
        GROUP BY c.complex_name;
    """
    return _run_query(sql)


def venues_in_country(country: str = 'Chile') -> pd.DataFrame:
    """
    3. Get details of venues in a specific country (default: 'Chile').
    """
    sql = """
        SELECT venue_name,
               city_name,
               country_name
        FROM Venues
        WHERE country_name = %s;
    """
    return _run_query(sql, (country,))


def list_venues_and_timezones() -> pd.DataFrame:
    """
    4. Identify all venues and their timezones.
    """
    sql = """
        SELECT venue_name,
               timezone
        FROM Venues;
    """
    return _run_query(sql)


def complexes_with_multiple_venues(min_count: int = 2) -> pd.DataFrame:
    """
    5. Find complexes that have more than one venue.
       (default threshold = 2)
    """
    sql = """
        SELECT c.complex_name,
               COUNT(v.venue_id) AS venue_count
        FROM Complexes AS c
        JOIN Venues    AS v
          ON c.complex_id = v.complex_id
        GROUP BY c.complex_name
        HAVING COUNT(v.venue_id) > %s;
    """
    return _run_query(sql, (min_count,))


def venues_grouped_by_country() -> pd.DataFrame:
    """
    6. List venues grouped by country.
    """
    sql = """
        SELECT country_name,
               COUNT(venue_id) AS venue_count
        FROM Venues
        GROUP BY country_name;
    """
    return _run_query(sql)


def venues_for_complex(complex_name: str = 'Nacional') -> pd.DataFrame:
    """
    7. Find all venues for a specific complex (default: 'Nacional').
    """
    sql = """
        SELECT v.venue_name
        FROM Venues    AS v
        JOIN Complexes AS c
          ON v.complex_id = c.complex_id
        WHERE c.complex_name = %s;
    """
    return _run_query(sql, (complex_name,))


if __name__ == "__main__":
    # Example usage — prints each DataFrame to console
    print("\n1. Venues with Complex Name:\n", list_venues_with_complex_name())
    print("\n2. Venue Count per Complex:\n", count_venues_per_complex())
    print("\n3. Venues in Chile:\n",       venues_in_country('Chile'))
    print("\n4. Venues & Timezones:\n",     list_venues_and_timezones())
    print("\n5. Complexes >1 Venue:\n",     complexes_with_multiple_venues(1))
    print("\n6. Venues by Country:\n",      venues_grouped_by_country())
    print("\n7. Venues for 'Nacional':\n",   venues_for_complex('Nacional'))



1. Venues with Complex Name:
            venue_name           complex_name
0      Cancha Central               Nacional
1      Cancha Central               Nacional
2        Centre Court  Estadio de la Cartuja
3        Centre Court  Estadio de la Cartuja
4           Court One  Estadio de la Cartuja
...               ...                    ...
13391         Court 2        Club Tennis Vic
13392    Centre Court        Club Tennis Vic
13393    Centre Court        Club Tennis Vic
13394         Court 3        Club Tennis Vic
13395         Court 3        Club Tennis Vic

[13396 rows x 2 columns]

2. Venue Count per Complex:
                                       complex_name  venue_count
0                          A.S.D. Tennis Club Como           36
1                                      AAJB Tennis           44
2                             Abama Tennis Academy           28
3                             Academy Zhangjiagang           28
4                 Acapulco Princess Mundo Imperial   

In [17]:
# 3.Competitor_Rankings Table and Competitors Table
import requests
import mysql.connector
# === 1. API Configuration ===
API_KEY = 'rltR6inycjP6oVTtY8lPaCd10gyZDchOu51tA3lo'
API_URL = f"https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key={API_KEY}"

# === 2. MySQL Connection ===
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
    database='tennis_analytics'
)
cursor = conn.cursor()

# === 3. Create Tables ===

cursor.execute("""
CREATE TABLE IF NOT EXISTS Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    abbreviation VARCHAR(10) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Competitor_Rankings (
    rank_id INT AUTO_INCREMENT PRIMARY KEY,
    rank INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50),
    FOREIGN KEY (competitor_id) REFERENCES Competitors(competitor_id)
);
""")

conn.commit()

# === 4. Fetch API Data ===
response = requests.get(API_URL)
data = response.json()

# === 5. Insert Data into Tables ===
for ranking in data.get('rankings', []):
    for competitor in ranking.get('competitor_rankings', []):
        comp = competitor.get('competitor', {})
        competitor_id = comp.get('id')
        name = comp.get('name')
        country = comp.get('country')
        country_code = comp.get('country_code')
        abbreviation = comp.get('abbreviation')

        if not all([competitor_id, name, country, country_code, abbreviation]):
            continue  # Skip if any essential information is missing

        # Insert into Competitors
        cursor.execute("""
            INSERT IGNORE INTO Competitors (competitor_id, name, country, country_code, abbreviation)
            VALUES (%s, %s, %s, %s, %s)
        """, (competitor_id, name, country, country_code, abbreviation))

        # Insert into Competitor_Rankings
        rank = competitor.get('rank')
        movement = competitor.get('movement')
        points = competitor.get('points')
        competitions_played = competitor.get('competitions_played')

        if not all([rank, movement, points, competitions_played]):
            continue  # Skip if any essential information is missing

        cursor.execute("""
            INSERT INTO Competitor_Rankings (rank, movement, points, competitions_played, competitor_id)
            VALUES (%s, %s, %s, %s, %s)
        """, (rank, movement, points, competitions_played, competitor_id))

conn.commit()

# === 6. Close Connection ===
cursor.close()
conn.close()


In [25]:
import mysql.connector

# === 1. Connect to MySQL Database ===
conn = mysql.connector.connect(host='localhost', user='root', password='', database='tennis_analytics')
cursor = conn.cursor()

# === 2. Execute Queries ===

# 1. Get all competitors with their rank and points
print("\n--- All Competitors with Rank and Points ---")
query = """
    SELECT c.name, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
"""
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# 2. Find competitors ranked in the top 5
print("\n--- Top 5 Ranked Competitors ---")
query = """
    SELECT c.name, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.rank <= 5
    ORDER BY cr.rank ASC
"""
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# 3. List competitors with no rank movement (stable rank)
print("\n--- Competitors with No Rank Movement ---")
query = """
    SELECT c.name, cr.rank, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.movement = 0
"""
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# 4. Get total points of competitors from a specific country (e.g., Croatia)
country = 'Croatia'
print(f"\n--- Total Points for Competitors from {country} ---")
query = """
    SELECT SUM(cr.points)
    FROM Competitor_Rankings cr
    JOIN Competitors c ON cr.competitor_id = c.competitor_id
    WHERE c.country = %s
"""
cursor.execute(query, (country,))
total = cursor.fetchone()[0]
print(f"Total Points: {total if total else 0}")

# 5. Count the number of competitors per country
print("\n--- Number of Competitors Per Country ---")
query = """
    SELECT country, COUNT(*) AS num_competitors
    FROM Competitors
    GROUP BY country
    ORDER BY num_competitors DESC
"""
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# 6. Find competitors with the highest points
print("\n--- Competitors with the Highest Points ---")
query = """
    SELECT c.name, cr.points
    FROM Competitors c
    JOIN Competitor_Rankings cr ON c.competitor_id = cr.competitor_id
    WHERE cr.points = (
        SELECT MAX(points) FROM Competitor_Rankings
    )
"""
cursor.execute(query)
results = cursor.fetchall()
for row in results:
    print(row)

# === 3. Close Connection ===
cursor.close()
conn.close()



--- All Competitors with Rank and Points ---
('Sachko, Vitaliy', 134, 591)
('Sachko, Vitaliy', 134, 591)
('Sachko, Vitaliy', 134, 591)
('Sachko, Vitaliy', 134, 591)
('Turker, Mert Naci', 473, 117)
('Turker, Mert Naci', 473, 117)
('Turker, Mert Naci', 473, 117)
('Turker, Mert Naci', 473, 117)
('Bolkvadze, Mariam', 403, 169)
('Bolkvadze, Mariam', 403, 169)
('Bolkvadze, Mariam', 403, 169)
('Bolkvadze, Mariam', 403, 169)
('Adams, Julia', 407, 166)
('Adams, Julia', 407, 166)
('Adams, Julia', 407, 166)
('Adams, Julia', 407, 166)
('Samson, Laura', 264, 305)
('Samson, Laura', 264, 305)
('Samson, Laura', 264, 305)
('Samson, Laura', 264, 305)
('Huang, Yujia', 359, 209)
('Huang, Yujia', 359, 209)
('Huang, Yujia', 359, 209)
('Huang, Yujia', 359, 209)
('Trhac, Patrik', 94, 872)
('Trhac, Patrik', 94, 872)
('Trhac, Patrik', 94, 872)
('Trhac, Patrik', 94, 872)
('Pawlikowska,  Zuzanna', 454, 143)
('Pawlikowska,  Zuzanna', 454, 143)
('Pawlikowska,  Zuzanna', 454, 143)
('Pawlikowska,  Zuzanna', 454, 143